In [2]:
# Dependencies 
import hvplot.pandas
import pandas as pd 
import requests 
from pathlib import Path 

# Import API key 
from api_keys import geoapify_key

In [8]:
# Load CSV filke created from WeatherPY into Pandas DataFrame
csv_file = Path("../WeatherPy/output_data/cities.csv")
city_data_df = pd.read_csv(csv_file)

city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,waitangi,-43.9535,-43.9535,9.29,58,100,1.78,NZ,1691388354
1,1,cidreira,-30.1811,-30.1811,18.73,73,41,2.14,BR,1691388355
2,2,papatowai,-46.5619,-46.5619,5.11,90,100,6.91,NZ,1691388356
3,3,udachny,66.4167,66.4167,19.14,48,98,3.80,RU,1691388357
4,4,sassandra,4.9500,4.9500,23.31,93,96,1.93,CI,1691388357


In [17]:
%%capture --no-display

# Configure map plot 
map_plot = city_data_df.hvplot.points(
    "Lng", 
    "Lat", 
    geo = True,
    tiles = "OSM", 
    frame_width = 1200, 
    frame_height = 600, 
    size = "Humidity", 
    color = "City", 
    alpha = 0.5
)

map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

In [18]:
# Narrow down cities that fit criteria and drop results with null values 
ideal_weather = city_data_df.loc[(city_data_df["Max Temp"] > 27) & (city_data_df["Max Temp"] < 28),
                                 ["City_ID", "City", "Lat", "Lng", "Max Temp", "Humidity", "Cloudiness", 
                                  "Wind Speed", "Country", "Date"]]

ideal_weather_df = ideal_weather.dropna(how='any')

ideal_weather_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
16,16,champerico,14.3000,14.3000,27.89,81,64,3.02,GT,1691388368
47,47,aleg,17.0481,17.0481,27.13,62,100,6.35,MR,1691388387
80,80,west island,-12.1568,-12.1568,27.99,78,75,5.66,CC,1691388409
123,123,newman,37.3138,37.3138,27.99,60,0,1.79,US,1691388439
126,126,kapa'a,22.0752,22.0752,27.18,81,75,6.69,US,1691388441
133,133,palikir - national government center,6.9248,6.9248,27.72,73,86,5.54,FM,1691388447
140,140,buala,-8.1450,-8.1450,27.36,79,65,1.98,SB,1691388451
141,141,lazaro cardenas,17.9583,17.9583,27.05,79,70,1.86,MX,1691388451
210,210,shimoda,34.6667,34.6667,27.84,74,15,5.06,JP,1691388497
215,215,oroqen zizhiqi,50.5667,50.5667,27.05,29,100,2.85,CN,1691388501


In [19]:
# Copy function to create hotel DataFrame
hotel_df = ideal_weather_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

hotel_df.loc[:, "Hotel Name"] = ""

hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
16,champerico,GT,14.3000,14.3000,81,
47,aleg,MR,17.0481,17.0481,62,
80,west island,CC,-12.1568,-12.1568,78,
123,newman,US,37.3138,37.3138,60,
126,kapa'a,US,22.0752,22.0752,81,


In [21]:
# Set parameters to search for hotel 
radius = 10000

params = {"categories": "accomodation.hotel",
          "limit": 20,
          "apiKey": geoapify_key}

print("Starting hotel search")

for index, row in hotel_df.iterrows():
    latitude = row["Lat"]
    longitude = row["Lng"]

    params["filter"]= f"circle:{longitude}, {latitude}, {radius}"
    params["bias"] = f"proximity:{longitude}, {latitude}"

    base_url = "https://api.geoapify.com/v2/places"

    name_address = requests.get(base_url, params=params) 
    name_address = name_address.json()

    try: 
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
    
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

hotel_df

Starting hotel search
champerico - nearest hotel: No hotel found
aleg - nearest hotel: No hotel found
west island - nearest hotel: No hotel found
newman - nearest hotel: No hotel found
kapa'a - nearest hotel: No hotel found
palikir - national government center - nearest hotel: No hotel found
buala - nearest hotel: No hotel found
lazaro cardenas - nearest hotel: No hotel found
shimoda - nearest hotel: No hotel found
oroqen zizhiqi - nearest hotel: No hotel found
shalqar - nearest hotel: No hotel found
moires - nearest hotel: No hotel found
turar ryskulov - nearest hotel: No hotel found
shingu - nearest hotel: No hotel found
contai - nearest hotel: No hotel found
aurangabad - nearest hotel: No hotel found
kanggye - nearest hotel: No hotel found
cambria - nearest hotel: No hotel found
rongelap - nearest hotel: No hotel found
salahonda - nearest hotel: No hotel found
gornyak - nearest hotel: No hotel found
puerto armuelles - nearest hotel: No hotel found
kailua-kona - nearest hotel: No hot

,City,Country,Lat,Lng,Humidity,Hotel Name
16,champerico,GT,14.3000,14.3000,81,No hotel found
47,aleg,MR,17.0481,17.0481,62,No hotel found
80,west island,CC,-12.1568,-12.1568,78,No hotel found
123,newman,US,37.3138,37.3138,60,No hotel found
126,kapa'a,US,22.0752,22.0752,81,No hotel found
133,palikir - national government center,FM,6.9248,6.9248,73,No hotel found
140,buala,SB,-8.1450,-8.1450,79,No hotel found
141,lazaro cardenas,MX,17.9583,17.9583,79,No hotel found
210,shimoda,JP,34.6667,34.6667,74,No hotel found
215,oroqen zizhiqi,CN,50.5667,50.5667,29,No hotel found


In [22]:
%%capture --no-display

# Configure map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    color = "City",
    hover_cols = ["Humidity","Hotel Name", "Country"],
    alpha = .5
)

# Display the map
map_plot_2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)